# Emulate v14

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load_hdf
from kepler.pandas.decorators import RingerDecorator
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy
import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

## Decorator:

In [3]:
def generator( df ):

    # for new training, we selected 1/2 of rings in each layer
    #pre-sample - 8 rings
    # EM1 - 64 rings
    # EM2 - 8 rings
    # EM3 - 8 rings
    # Had1 - 4 rings
    # Had2 - 4 rings
    # Had3 - 4 rings
    prefix = 'trig_L2_cl_ring_%i'

    # rings presmaple 
    presample = [prefix %iring for iring in range(8//2)]
    # EM1 list
    sum_rings = 8
    em1 = [prefix %iring for iring in range(sum_rings, sum_rings+(64//2))]
    # EM2 list
    sum_rings = 8+64
    em2 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # EM3 list
    sum_rings = 8+64+8
    em3 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # HAD1 list
    sum_rings = 8+64+8+8
    had1 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD2 list
    sum_rings = 8+64+8+8+4
    had2 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD3 list
    sum_rings = 8+64+8+8+4+4
    had3 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    col_names = presample+em1+em2+em3+had1+had2+had3
    rings = df[col_names].values.astype(np.float32)

    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    data_rings = norm1(rings)
    return [data_rings]

In [4]:
tpath = 'output/fitting'

In [5]:
decorators = [
    RingerDecorator('ringer_v14_tight' , tpath + '/ElectronRingerTightTriggerConfig.conf'    , generator),
    RingerDecorator('ringer_v14_medium', tpath + '/ElectronRingerMediumTriggerConfig.conf'   , generator),
    RingerDecorator('ringer_v14_loose' , tpath + '/ElectronRingerLooseTriggerConfig.conf'    , generator),
    RingerDecorator('ringer_v14_vloose', tpath + '/ElectronRingerVeryLooseTriggerConfig.conf', generator),
]

2022-03-05 11:19:53,535 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerTightTriggerConfig.conf
2022-03-05 11:19:55,697 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerMediumTriggerConfig.conf
2022-03-05 11:19:57,481 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerLooseTriggerConfig.conf
2022-03-05 11:19:59,176 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerVeryLooseTriggerConfig.conf


2022-03-05 11:19:53.735953: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-03-05 11:19:53.735993: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-03-05 11:19:53.736018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba51): /proc/driver/nvidia/version does not exist
2022-03-05 11:19:53.738934: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 11:19:53.831497: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-03-05 11:19:53.839517: I tensorflow/compiler/xla/service/service.c

In [6]:
def load_in_loop( paths, decorators ):

    tables = []
    for path in progressbar( paths, prefix='Reading files...'):
        df = load_hdf( path )
        hold_these_columns = []
        for decorator in decorators:
            decorator.apply(df) 
            hold_these_columns.append(decorator.column)
            hold_these_columns.append(decorator.column + '_output')
            
        tables.append(df[hold_these_columns])

    return pd.concat(tables).reset_index(drop=True)

## Read all 2017 bins:

In [7]:
dpath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
dpath+= '/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [8]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [30:40<00:00, 73.61s/it] 


In [9]:
my_df.to_hdf(output_path+'/data17_table_v14.h5', key='df', mode='w')

## Read all 2018 bins:

In [10]:
dpath = '/home/jodafons/public/cern_data/new_files/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
dpath+= '/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [11]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [32:13<00:00, 77.34s/it] 


In [12]:
my_df.to_hdf(output_path+'/data18_table_v14.h5', key='df', mode='w')

## Read all mc16 boosted bins:

In [13]:
dpath = '/home/jodafons/public/cern_data/new_files/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2'
dpath+= '/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [14]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [00:10<00:00,  2.47it/s]


In [15]:
my_df.to_hdf(output_path+'/mc16_table_v14.h5', key='df', mode='w')